In [1]:
import cv2

import numpy as np
import matplotlib.pyplot as plt

import os, json
import random, traceback

import time

import shutil

import torch
from torch import nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
class Faces(object):
    
    def __init__(self, face_cascade, face_dir, output_dir,):
        self.face_cascade = face_cascade
        self.image_num = 1
        self.dir_num = 1
        self.face_dir = face_dir
        self.output_dir = output_dir
        self.image_name = 'Face'
        self.error = []
        
    def face_co_ordinates(self, gray):
        return self.face_cascade.detectMultiScale(gray, 1.2, 4)
#         return face_classifier.detectMultiScale(gray, 1.2, 4)
    
    def draw_rectangle(self, image, co_ordinate):
        x,y,w,h = co_ordinate
        cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 2)
        return None
    
    def crop_face(self, image, co_ordinate):
        x,y,w,h = co_ordinate
        return image[y:y+h, x:x+w]
    
    def all_faces(self, draw_face=False):
        images = os.listdir(self.face_dir)
        print(self.dir_num, self.face_dir,'  starts with: ', self.image_num,'  having images:', len(images))
        for image_name in images:
            if image_name[-4:] == '.jpg':
                image = cv2.imread(os.path.join(self.face_dir, image_name))
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                co_ordinates = self.face_co_ordinates(gray)

                if len(co_ordinates) > 0:
                    for co_ordinate in co_ordinates:
                        if draw_face:
                            self.draw_rectangle(image, co_ordinate)
                            cv2.imshow('Sunny', image)
                            cv2.waitKey(500)
                        else:
                            try:
                                crop_image = self.crop_face(image, co_ordinate)
                                name = os.path.join(self.output_dir, self.image_name + str(self.image_num) +'.jpg')
                                cv2.imwrite(name, crop_image)
                                self.image_num += 1
                            except:
                                self.error.append(image_name)
                                continue
        print('No. of errors: ', len(self.error))
        print('====================================================================================================')
        data = {'Errors': self.error}
        df = pd.DataFrame(data)
        img_dir = self.face_dir.replace('./IMAGES/', '')
        err_file = os.path.join(self.output_dir, 'error_'+img_dir[:-1]+'.csv')
        df.to_csv(err_file)
        cv2.destroyAllWindows()
        return None

In [3]:
classifier_path = '/media/amit/CA301AE8301ADAF1/Documents/PROJECT/Face/HAARCASCADE/'
classifier = os.listdir(classifier_path)
face_classifier = cv2.CascadeClassifier(os.path.join(classifier_path, classifier[0]))

In [4]:
img_path = "/media/amit/CA301AE8301ADAF1/Documents/PROJECT/GARMENTS/Vids"
imgs = os.listdir(img_path)
imgs

['Manish Malhotra Label _ Zween Couture, Doha 2018_19.mp4',
 'Suneet Varma _ Fall_Winter 2019_20 _ India Fashion Week.mp4',
 'Tarun Tahiliani - FDCI x Lakmé Fashion Week.mp4',
 'y2mate.com - Chanel  Fall Winter 20222023  Full Show_1080p.mp4',
 'y2mate.com - Emporio Armani FallWinter 202223 Milan_1080p.mp4',
 'y2mate.com - Georges Hobeika  Haute Couture Fall Winter 20172018 Full Show  Exclusive_1080p.mp4',
 'y2mate.com - Georges Hobeika  Spring Summer 2022  Full Show_1080p.mp4',
 'y2mate.com - Luli Fama Swimwear Fashion Show SS 2019 Miami Swim Week 2018 Paraiso Fashion Fair_1080p.mp4',
 'y2mate.com - Luli Fama Swimwear Fashion Show SS2020 Miami Swim Week 2019 Paraiso Miami Beach_1080p.mp4',
 'y2mate.com - Miu Miu  Fall Winter 20222023  Full Show_1080p.mp4',
 'y2mate.com - Oh Polly presents Neena Swim Bikini Fashion Show 2022 SS22 Miami Swim Week _1080p.mp4',
 'y2mate.com - ZUHAIR MURAD  Couture Fall Winter 20212022 Show_1080p.mp4',
 'y2mate.com - Zuhair Murad  Haute Couture Spring Summe

In [5]:
class NodeCNN(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=4, kernel_size=3,stride=1,padding=1,dilation=1,groups=1,bias=True,)
        self.conv2 = nn.Conv2d(in_channels=4,out_channels=16, kernel_size=3,stride=1,padding=1,dilation=1,groups=1,bias=True,)
        # self.conv2 = nn.Conv2d(in_channels=8,out_channels=16, kernel_size=3,stride=1,padding=1,dilation=1,groups=1,bias=True,)
        
        self.fc1 = nn.Linear(2*2*16*16, 128)
        self.fc2 = nn.Linear(128, 16)
        self.fc3 = nn.Linear(16, 2)
        
    def forward(self, X):
        
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2,2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2,2)
        # X = F.relu(self.conv3(X))
        # X = F.max_pool2d(X, 2,2)
        X = X.view(-1, 16*16*2*2)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        
        return F.log_softmax(X, dim=1)

In [11]:
model = NodeCNN()

In [12]:
model_path = '/media/amit/CA301AE8301ADAF1/Documents/PROJECT/Face/FACE_CLASSIFIER_MODEL_2'

test_model_path = os.path.join(model_path, "FRS_CLASSIFIER_Model_50.net")
test_model = model.load_state_dict(torch.load(test_model_path, map_location=torch.device('cpu')))
test_model

<All keys matched successfully>

In [13]:
model.eval()

NodeCNN(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [14]:
class Faces(object):
    
    def __init__(self, face_cascade, face_dir, output_dir,):
        self.face_cascade = face_cascade
        self.image_num = 1
        self.dir_num = 1
        self.face_dir = face_dir
        self.output_dir = output_dir
        self.image_name = 'Face'
        self.error = []
        
    def face_co_ordinates(self, gray):
        return self.face_cascade.detectMultiScale(gray, 1.2, 4)
#         return face_classifier.detectMultiScale(gray, 1.2, 4)

    def neck_cordinates(self, x,y,w,h):
        return int(x+(w/8)), (y+h), int(w-(w/4)), int(h*0.4)
    
    def upper_half_torso_cordinates(self, x,y,w,h, h1):
        return int(x - (w/2)), (y+h+h1), 2*w, 2*h
        
    def lower_half_torso_cordinates(self, x,y,w,h, h_):
        return x, y+h, w, h_
    
    def legs_cordinates(self, x,y,w,h, h_):
        return x, y+h, w, 4*h_
    
    def foot_cordinates(self, x,y,w,h, h_):
        return x, y+h, w, int(h_/2)
    
    def colors(self):
        return (random.choice(range(0, 255)), random.choice(range(0, 255)), random.choice(range(0, 255)))
    
    def draw_rectangle(self, image, co_ordinate, color):
        x,y,w,h = co_ordinate
        cv2.rectangle(image, (x,y), (x+w, y+h), color, 2)
        return None
    
    def crop_face(self, image, co_ordinate):
        x,y,w,h = co_ordinate
        return image[y:y+h, x:x+w]
    
    def all_faces(self, draw_face=False):
        images = os.listdir(self.face_dir)
        print(self.dir_num, self.face_dir,'  starts with: ', self.image_num,'  having images:', len(images))
        for image_name in images:
            if image_name[-4:] == '.jpg':
                image = cv2.imread(os.path.join(self.face_dir, image_name))
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                co_ordinates = self.face_co_ordinates(gray)

                if len(co_ordinates) > 0:
                    for co_ordinate in co_ordinates:
                        if draw_face:
                            x,y,w,h = co_ordinate
                            self.draw_rectangle(image, co_ordinate)
                            x1, y1, w1, h1 = self.neck_cordinates(x,y,w,h)
                            self.draw_rectangle(image, (x1, y1, w1, h1))
                            x2, y2, w2, h2 = self.upper_half_torso_cordinates(x,y,w,h,h1)
                            self.draw_rectangle(image, (x2, y2, w2, h2))
                            x3, y3, w3, h3 = self.lower_half_torso_cordinates(x3, y3, w3, h3, h)
                            self.draw_rectangle(image, (x3, y3, w3, h3))
                            x4, y4, w4, h4 = self.legs_cordinates(x3, y3, w3, h3, h_)
                            self.draw_rectangle(image, (x4, y4, w4, h4))
                            x5, y5, w5, h5 = self.foot_cordinates(x4, y4, w4, h4, h_)
                            self.draw_rectangle(image, (x5, y5, w5, h5))
                            cv2.imshow('Sunny', image)
                            cv2.waitKey(500)
                        else:
                            break
#                             try:
#                                 crop_image = self.crop_face(image, co_ordinate)
#                                 name = os.path.join(self.output_dir, self.image_name + str(self.image_num) +'.jpg')
#                                 cv2.imwrite(name, crop_image)
#                                 self.image_num += 1
#                             except:
#                                 self.error.append(image_name)
#                                 continue
        print('No. of errors: ', len(self.error))
        print('====================================================================================================')
        data = {'Errors': self.error}
        df = pd.DataFrame(data)
        img_dir = self.face_dir.replace('./IMAGES/', '')
        err_file = os.path.join(self.output_dir, 'error_'+img_dir[:-1]+'.csv')
        df.to_csv(err_file)
        cv2.destroyAllWindows()
        return None

In [15]:
class VideoFace(Faces):
    
    def __init__(self, video_path, face_cascade, face_dir, output_dir):
        Faces.__init__(self, face_cascade, face_dir, output_dir)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        self.video = video_path
        self.video_num = 1
        self.is_face = False
        
    def get_video_face(self, draw_face=False):
        
        cap = cv2.VideoCapture(self.video)
        print(self.video_num, 'Video Running -->', self.video)
        while True:
            self.is_face = False
            success, image = cap.read()
            
            if success:
                image = cv2.resize(image, (640, 360))
                try:
                    image = np.uint8(image)
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                except:
                    break
#                 print(self.image_num)
                co_ordinates = self.face_co_ordinates(gray)

                if len(co_ordinates) > 0:
                    for co_ordinate in co_ordinates:
                        if draw_face:
                            color = self.colors()
                            x,y,w,h = co_ordinate
                            try:
                                crop_image = self.crop_face(image, co_ordinate)
                                name = os.path.join(self.output_dir, self.image_name + str(self.image_num) +'.jpg')
                                cv2.imwrite(name, crop_image)
                                self.image_num += 1
                            except:
                                print(traceback.print_exc(), exc_info=True)
                            try:
                                x_ = cv2.resize(cv2.cvtColor(crop_image, cv2.COLOR_BGR2GRAY), (2*16, 32))/255
                                x_ = x_.astype(np.float32)
                                x_ = torch.tensor(x_)
                                x_ = x_.view((1,1,32,16*2))
                                pred = model(x_)

                                pred = int(pred.argmax().item())
                                
                                if pred == 1:
                                    print("true face detected ----------- ")
                                    self.is_face = True
                                    
                            except:
                                print(traceback.print_exc(), exc_info=True)
                                
                            if self.is_face:
                                self.draw_rectangle(image, co_ordinate, color)
                                x1, y1, w1, h1 = self.neck_cordinates(x,y,w,h)
                                self.draw_rectangle(image, (x1, y1, w1, h1), color)
                                x2, y2, w2, h2 = self.upper_half_torso_cordinates(x,y,w,h,h1)
                                self.draw_rectangle(image, (x2, y2, w2, h2), color)
    #                             print("888888888888888888888888")
                                x3, y3, w3, h3 = self.lower_half_torso_cordinates(x2, y2, w2, h2, h)
                                self.draw_rectangle(image, (x3, y3, w3, h3), color)
                                x4, y4, w4, h4 = self.legs_cordinates(x3, y3, w3, h3, h)
                                self.draw_rectangle(image, (x4, y4, w4, h4), color)
    #                             print("999999999999999999999999")
                                x5, y5, w5, h5 = self.foot_cordinates(x4, y4, w4, h4, h)
                                self.draw_rectangle(image, (x5, y5, w5, h5), color)
    #                             self.image_num += 1
                
                cv2.imshow('Fashion Show', image)
    #                         cv2.waitKey(1)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                break

        cap.release()
        cv2.destroyAllWindows()
        return None

In [16]:
def get_all_video_faces():
#     classifier_path = "/media/amit/CA301AE8301ADAF1/Documents/PROJECT/Face/HAARCASCADE/"
    classifier = os.listdir(classifier_path)
    face_classifier = cv2.CascadeClassifier(os.path.join(classifier_path, classifier[0]))
    
    images_dir_path = './IMAGES/'
    output_dir = '/media/amit/CA301AE8301ADAF1/Documents/PROJECT/GARMENTS/Fashion_Show/FACE_DATA33'
    video_dir_path = img_path
    image_num = 1
    dir_num = 1
    vid_num = 1
    images_path = './None'
    
    video_dir = os.listdir(video_dir_path)
    
    for video in video_dir[1:]:
        video_path = os.path.join(video_dir_path, video)
        output_dir1 = os.path.join(output_dir, os.path.splitext(video)[0])
        vf = VideoFace(video_path, face_classifier, images_path, output_dir1)
        vf.image_num = image_num
        vf.vid_num = vid_num
        vf.get_video_face(draw_face=True)
        
        image_num = vf.image_num
        vid_num += 1
    cv2.destroyAllWindows()
    return None

In [17]:
get_all_video_faces()

1 Video Running --> /media/amit/CA301AE8301ADAF1/Documents/PROJECT/GARMENTS/Vids/Suneet Varma _ Fall_Winter 2019_20 _ India Fashion Week.mp4
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected --------

true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true fac

true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true fac

true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true fac

true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true fac

...

true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true fac

true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true fac

true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true fac

true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true face detected ----------- 
true fac

true face detected ----------- 
true face detected ----------- 
1 Video Running --> /media/amit/CA301AE8301ADAF1/Documents/PROJECT/GARMENTS/Vids/y2mate.com - Georges Hobeika  Haute Couture Fall Winter 20172018 Full Show  Exclusive_1080p.mp4
1 Video Running --> /media/amit/CA301AE8301ADAF1/Documents/PROJECT/GARMENTS/Vids/y2mate.com - Georges Hobeika  Spring Summer 2022  Full Show_1080p.mp4
1 Video Running --> /media/amit/CA301AE8301ADAF1/Documents/PROJECT/GARMENTS/Vids/y2mate.com - Luli Fama Swimwear Fashion Show SS 2019 Miami Swim Week 2018 Paraiso Fashion Fair_1080p.mp4
1 Video Running --> /media/amit/CA301AE8301ADAF1/Documents/PROJECT/GARMENTS/Vids/y2mate.com - Luli Fama Swimwear Fashion Show SS2020 Miami Swim Week 2019 Paraiso Miami Beach_1080p.mp4
1 Video Running --> /media/amit/CA301AE8301ADAF1/Documents/PROJECT/GARMENTS/Vids/y2mate.com - Miu Miu  Fall Winter 20222023  Full Show_1080p.mp4
1 Video Running --> /media/amit/CA301AE8301ADAF1/Documents/PROJECT/GARMENTS/Vids/y2mate.com -

In [ ]:
path = './'
os.listdir(path)

In [ ]:
def get_all_faces():
    classifier_path = './HAARCASCADE/'
    classifier = os.listdir(classifier_path)
    face_classifier = cv2.CascadeClassifier(os.path.join(classifier_path, classifier[0]))
    
    images_dir_path = './MY/'
    output_dir = './FACES2/'
    image_num = 50000
    dir_num = 1
    
    images_dirs = os.listdir(images_dir_path)
    
    for image_dir in images_dirs:
        images_path = os.path.join(images_dir_path, image_dir)
        fc = Faces(face_classifier, images_path, output_dir)
        fc.image_num = image_num
        fc.dir_num = dir_num
        fc.all_faces()
        image_num = fc.image_num
        dir_num += 1
        

In [ ]:
# get_all_faces()

In [ ]:
final_face_path = './FINAL-FACES/'

In [ ]:
final_face_images = os.listdir(final_face_path)

In [ ]:
imgs_name  = []
imgs_shape = []
for i in final_face_images:
    img = cv2.imread(os.path.join(final_face_path, i))
    imgs_name.append(i)
    imgs_shape.append(img.shape[0])

In [ ]:
plt.plot(np.arange(len(imgs_shape)), sorted(imgs_shape))

In [ ]:
data = {
    'image': imgs_name,
    'shape': imgs_shape,
}

df = pd.DataFrame(data)

In [ ]:
x = np.array(imgs_shape)

In [ ]:
x[x>1000]

In [ ]:
df[df['shape']>1000]

In [ ]:
sunny_img = cv2.imread(os.path.join(sunny_images_path, 'Image99.jpg'))

In [ ]:
for i in sunny_images[51:200]:
    sunny_img = cv2.imread(os.path.join(sunny_images_path, i))
    
    gray = cv2.cvtColor(sunny_img, cv2.COLOR_BGR2GRAY)
    
    faces = face_classifier.detectMultiScale(gray, 1.1, 4)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(sunny_img, (x,y), (x+w, y+h), (0, 255,0), 3)
    cv2.imshow('sunny', sunny_img)
    cv2.waitKey(1500)
cv2.destroyAllWindows()

In [ ]:
sunny_img.shape

In [ ]:
x,y,w,h

In [ ]:
raw = sunny_img[y:y+h, x:x+w]

In [ ]:
plt.imshow(raw)

In [ ]:
plt.imshow(sunny_img)

In [ ]:
p = 'k/l/m/'

In [ ]:
os.path.basename(p)

In [ ]:
plt.imshow(sunny_img)

In [ ]:
gray = cv2.cvtColor(sunny_img, cv2.COLOR_BGR2GRAY)
plt.imshow(gray)
# gray = cv2.flip(gray, 1)

In [ ]:
faces = face_classifier.detectMultiScale(gray, 1.1, 4)

In [ ]:
faces

In [ ]:
img = sunny_img.copy()
# img = cv2.flip(img, 1)
plt.imshow(img)

In [ ]:
for (x,y,w,h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255,0), 3)

In [ ]:
plt.imshow(img)

In [ ]:
raw = img[y:y+h, x:x+w]

In [ ]:
plt.imshow(raw)